In [173]:
import pandas as pd
import plotly.express as px
import numpy as np
import os

In [174]:
prepend = 'csv_exports/nft_sales/'
filen = 'marketplace_stats_'
date = '23_01_23'
mktpl_df = pd.read_csv(prepend + filen + date + '.csv')
mktpl_df['dt'] = pd.to_datetime(mktpl_df['dt']).dt.tz_localize(None)

# print(mktpl_df.dtypes)


In [175]:
min_date = pd.to_datetime('2023-01-18', format='%Y-%m-%d')
# print(min_date)

mktpl_df = mktpl_df[(mktpl_df['dt'] >= pd.to_datetime(min_date) ) \
                        & (mktpl_df['dt'] < pd.to_datetime("today") )\
                        ]

total_df = mktpl_df.copy()
total_df['marketplace'] = 'Total' #override for sums

#add extra "totals" row
total_df = pd.concat([mktpl_df,total_df])

In [180]:
# segment by marketplace

mktpl_agg = total_df.groupby(['marketplace']).agg({
    'num_trades':'sum'
    ,'eth_value_traded':'sum'
    ,'usd_value_traded':'sum'
    ,'dt':'nunique'
})


mktpl_agg.reset_index(inplace=True)
mktpl_agg = mktpl_agg.rename(columns={'dt':'num_days'})
display(mktpl_agg)

for c in mktpl_agg.columns:
    if c == 'marketplace':
        continue
    else:
        mktpl_agg[c+'_avg'] = mktpl_agg[c] / mktpl_agg['num_days']
        mktpl_agg[c+'_annualized'] = ( mktpl_agg[c] / mktpl_agg['num_days'] ) * 365
        
        total_num = mktpl_agg.loc[mktpl_agg['marketplace'] == 'Total'][c].iloc[0]
        mktpl_agg[c+'_pct_share'] = mktpl_agg[c] / total_num

display(mktpl_agg)

,marketplace,num_trades,eth_value_traded,usd_value_traded,num_days
0,OpenSea,3384,23.590396,37424.347530,5
1,Quix,3749,13.952091,22290.841571,5
2,Total,7133,37.542488,59715.189101,5


7133
37.542487617821436
59715.18910110811
5


,marketplace,num_trades,eth_value_traded,usd_value_traded,num_days,num_trades_avg,num_trades_annualized,eth_value_traded_avg,eth_value_traded_annualized,usd_value_traded_avg,usd_value_traded_annualized,num_days_avg,num_days_annualized
0,OpenSea,3384,23.590396,37424.347530,5,676.8,247032.0,4.718079,1722.098922,7484.869506,2.731977e+06,1.0,365.0
1,Quix,3749,13.952091,22290.841571,5,749.8,273677.0,2.790418,1018.502674,4458.168314,1.627231e+06,1.0,365.0
2,Total,7133,37.542488,59715.189101,5,1426.6,520709.0,7.508498,2740.601596,11943.037820,4.359209e+06,1.0,365.0
